# 모듈

In [24]:
import pandas as pd
import numpy as np  

# 데이터 전처리

## 미사용 컬럼 제거

### 서울 공중화장실

In [25]:
pub = pd.read_csv('공중화장실.csv', encoding='cp949')
pub.columns

Index(['고유번호', '구명', '법정동명', '산지여부', '주지번', '부지번', '새주소명', '생성일', '위도', '경도'], dtype='object')

In [26]:
pub_og = pub.copy()   # 원본 복사

In [27]:
# 미사용 컬럼 제거

pub = pub[['구명', '법정동명','위도', '경도']]
pub.head(2)

,구명,법정동명,위도,경도
0,송파구,마천동,37.501401,127.158647
1,노원구,하계동,37.644937,127.073728


### 지하철 화장실

In [29]:
sub = pd.read_csv('지하철화장실.csv',encoding='cp949')   # 
sub.columns

Index(['연번 ', '구분 ', '관리기관명', '운영노선명 ', '역명', '소재지도로명주소 ', '소재지지번주소 ', '위도 ',
       '경도 ', '지상 또는 지하 구분', '역층', '게이트 내외 구분', '(근접) 출입구 번호', '상세위치 ',
       '전화번호 ', '개방시간 ', '화장실 설치장소 유형', '오물처리방식 ', '남녀공용화장실여부 ', '남성용-대변기수 ',
       '남성용-소변기수 ', '남성용-장애인용대변기수 ', '남성용-장애인용소변기수 ', '남성용-어린이용대변기수 ',
       '남성용-어린이용소변기수 ', '여성용-대변기수 ', '여성용-장애인용대변기수 ', '여성용-어린이용대변기수 ',
       '비상벨 설치유무 ', '화장실입구cctv설치유무 ', '기저귀교환대설치유무-남자화장실 ',
       '기저귀교환대설치유무-남자장애인화장실 ', '기저귀교환대설치유무-여자화장실 ', '기저귀교환대설치유무-여자장애인화장실 ',
       '리모델링 연도', '데이터기준일자'],
      dtype='object')

In [30]:
sub_og = sub.copy()   # 원본 복사

In [31]:
# 미사용 컬럼 제거

sub = sub[['운영노선명 ', '역명', '위도 ', '경도 ', '게이트 내외 구분', '리모델링 연도']]
sub.head(2)      

,운영노선명,역명,위도,경도,게이트 내외 구분,리모델링 연도
0,1호선,서울,37.556878,126.972578,외부,2015
1,1호선,시청,37.565682,126.976849,외부,2013


In [32]:
sub.isnull().sum() # 불러오는 과정에서 생성된 공백 행 10개 제거

운영노선명        10
역명           10
위도           10
경도           10
게이트 내외 구분    10
리모델링 연도      10
dtype: int64

In [33]:
sub = sub.dropna()

In [34]:
sub = sub.rename(columns={'운영노선명 ': '운영노선명', '위도 ': '위도', '경도 ': '경도', '리모델링연도 ':'리모델링연도'})

## 필요 컬럼 추가
- 서울 **공중화장실** 위치 정보 데이터에 **가까운 역** 컬럼 추가

In [35]:
# Haversine

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반경의 키로미터 단위
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# 각 공중화장실으로부터 직선거리가 가장 가까운 지하철 역 찾는 함수
def find_closest_station(toilet_lat, toilet_lon, stations):
    distances = stations.apply(
        lambda row: haversine_distance(toilet_lat, toilet_lon, row['위도'], row['경도']), axis=1
    )
    closest_station = stations.loc[distances.idxmin()]
    return closest_station['역명']

# Apply the function to each toilet location
pub.loc[:, '가까운역'] = pub.apply(
    lambda row: find_closest_station(row['위도'], row['경도'], sub), axis=1
)

In [36]:
# 결과 확인
pub.head(6)

,구명,법정동명,위도,경도,가까운역
0,송파구,마천동,37.501401,127.158647,마천
1,노원구,하계동,37.644937,127.073728,노원
2,노원구,하계동,37.640335,127.077637,중계
3,강북구,수유동,37.645724,127.020449,노원
4,도봉구,방학동,37.659783,127.031187,상계
5,송파구,가락동,37.502339,127.123028,오금


In [37]:
pub.tail(6)  # 이상 무

,구명,법정동명,위도,경도,가까운역
5040,강서구,등촌1동,37.555880,126.858918,발산
5041,강서구,화곡2동,37.531873,126.854461,신정
5042,강서구,우장산동,37.553483,126.843646,발산
5043,강서구,등촌동,37.558864,126.848219,발산
5044,강서구,등촌동,37.548020,126.860020,화곡
5045,강서구,화곡동,37.551076,126.849544,우장산


## 노이즈 데이터 처리

### 서울 공중화장실 

In [41]:
# "구" 명이 잘못 기입된 데이터들 수정
pub.loc[pub['구명'] == '영등포', '구명'] = '영등포구'
pub.loc[pub['구명'] == '영등로구', '구명'] = '영등포구'
pub.loc[pub['구명'] == '영등표구', '구명'] = '영등포구'
pub.loc[pub['구명'] == '금천', '구명'] = '금천구'
pub.loc[pub['구명'] == '강서', '구명'] = '강서구'
pub.loc[pub['구명'] == '노원', '구명'] = '노원구'
pub.loc[pub['구명'] == '동작', '구명'] = '동작구'
pub.loc[pub['구명'] == '관악', '구명'] = '관악구'
pub.loc[pub['구명'] == '송파ㅜ', '구명'] = '송파구'
pub.loc[pub['구명'] == '갈현송방차풀소', '구명'] = '은평구'
pub.loc[pub['구명'] == '송북구', '구명'] = '성북구'
pub.loc[pub['구명'] == '송파두성빌딩', '구명'] = '송파구'
pub.loc[pub['구명'] == '갈암구', '구명'] = '강남구'
pub.loc[pub['구명'] == '구로수', '구명'] = '구로구'
pub.loc[pub['구명'] == '남서울빌딩', '구명'] = '영등포구'
pub.loc[pub['구명'] == '서대문', '구명'] = '서대문구'
pub.loc[pub['구명'] == '양천', '구명'] = '양천구'

,구명,법정동명,위도,경도,가까운역
0,송파구,마천동,37.501401,127.158647,마천
1,노원구,하계동,37.644937,127.073728,노원
2,노원구,하계동,37.640335,127.077637,중계
3,강북구,수유동,37.645724,127.020449,노원
4,도봉구,방학동,37.659783,127.031187,상계


In [ ]:
pub['구명'].unique() # 최종 확인인

In [ ]:
pub.to_csv('public_rest_gu.csv', encoding='cp949', index=False)

### 지하철 화장실

In [45]:
sub['리모델링 연도'].unique()

array(['2015', '2013', ' 2008이전 ', '2010', '2014', '2016', '2009', '2012',
       '2022', '2017', '2018', '2011', '2019', '2020', '2024', '2021',
       '2008', '2006', '2008이전'], dtype=object)

In [46]:
# 리모델링 연도 '2008년이전' 내역을 2006 으로 일괄 수정

sub.loc[(sub['리모델링 연도'] == '2008이전') | (sub['리모델링 연도'] == ' 2008이전 '), '리모델링 연도'] = 2006

In [47]:
sub['리모델링 연도'].unique() # 최종확인

array(['2015', '2013', 2006, '2010', '2014', '2016', '2009', '2012',
       '2022', '2017', '2018', '2011', '2019', '2020', '2024', '2021',
       '2008', '2006'], dtype=object)

In [48]:
sub['리모델링 연도'] = sub['리모델링 연도'].astype('int')

In [ ]:
# 게이트 내외 구분값이 내부인 경우 제거

sub = sub[sub['게이트 내외 구분']=='외부']

In [51]:
# 하나의 지하철 역에 화장실 개수가 많은 케이스 중, 리모델링 년수까지 동일한 행 제거
sub = sub[sub.duplicated()==0]

In [52]:
sub.to_csv('subway_rest_del.csv', encoding='cp949', index=False) 